# Skip-gram avec l'échantillonnage négatif :
Skip-gram est un modèle qui vise à prédire les mots contextuels donnés un mot cible. L'échantillonnage négatif est une technique utilisée pour rendre l'entraînement plus efficace en échantillonnant des exemples "négatifs" (mots qui ne font pas partie du contexte) ainsi que les vrais exemples positifs (mots faisant partie du contexte). L'objectif est de maximiser la probabilité d'observer les vrais mots contextuels tout en minimisant la probabilité d'observer des mots négatifs échantillonnés au hasard.

Le modèle skip-gram définit deux ensembles de vecteurs de mots :
1. Les vecteurs de mots d'entrée (cibles), notés $v(w_i)$ pour le mot cible $w_i$.
2. Les vecteurs de mots de sortie (contextuels), notés $u(w_j)$ pour le mot contextuel $w_j$.

La probabilité d'observer un mot contextuel $w_j$ étant donné un mot cible $w_i$ est modélisée à l'aide de la fonction sigmoïde :

$$
P(D = 1 | w_i, w_j) = \sigma(u(w_j) \cdot v(w_i))
$$

Où $\sigma(x)$ est la fonction sigmoïde :

$$
\sigma(x) = \frac{1}{1 + e^{-x}}
$$

Pour entraîner le modèle, on cherche à maximiser la vraisemblance d'observer des mots contextuels lorsque l'on a un exemple positif (un vrai mot contextuel) et à la minimiser pour les exemples négatifs échantillonnés au hasard.

La fonction de perte pour un seul exemple positif (mot contextuel) et $K$ exemples négatifs est définie comme suit :

$$
L = -\log(P(D = 1 | w_i, w_j)) - \sum_{k=1}^K \log(P(D = 0 | w_i, w_k))
$$

Où $P(D = 0 | w_i, w_k)$ est la probabilité d'observer un mot exemple négatif $w_k$.

Optimisation par descente de gradient :
Pour optimiser le modèle Word2Vec en utilisant la descente de gradient, vous devez calculer les gradients de la fonction de perte par rapport aux vecteurs de mots $u(w_j)$ et $v(w_i)$, puis mettre à jour ces vecteurs en conséquence.

1. Calculez les gradients par rapport à $u(w_j)$ et $v(w_i)$ :
   - Gradient pour un exemple positif :

     $ \nabla u(w_j) = \frac{\partial L}{\partial u(w_j)} = (P(D = 1 | w_i, w_j) - 1) \cdot v(w_i)$
     
     $\nabla v(w_i) = \frac{\partial L}{\partial v(w_i)} = (P(D = 1 | w_i, w_j) - 1) \cdot u(w_j)$

   - Gradient pour les exemples négatifs ($w_k$) :

     $\nabla u(w_k) = \frac{\partial L}{\partial u(w_k)} = -P(D = 0 | w_i, w_k) \cdot v(w_i) $

     $\nabla v(w_i) = \frac{\partial L}{\partial v(w_i)} = -P(D = 0 | w_i, w_k) \cdot u(w_k)$

2. Mettez à jour les vecteurs de mots en utilisant les gradients et un taux d'apprentissage ($\alpha$) :
   - Règle de mise à jour pour $u(w_j)$ et $v(w_i)$ pour un exemple positif :

     $u(w_j) \leftarrow u(w_j) - \alpha \cdot \nabla u(w_j)$

     $v(w_i) \leftarrow v(w_i) - \alpha \cdot \nabla v(w_i)$

   - Règle de mise à jour pour $u(w_k)$ et $v(w_i)$ pour les exemples négatifs ($w_k$) :

     $u(w_k) \leftarrow u(w_k) - \alpha \cdot \nabla u(w_k)$

     $v(w_i) \leftarrow v(w_i) - \alpha \cdot \nabla v(w_i)$

3. Répétez ce processus sur plusieurs itérations (époques) et sur l'ensemble de votre jeu de données pour entraîner le modèle Word2Vec.

La descente de gradient permet au modèle d'apprendre des embeddings de mots qui capturent les relations sémantiques entre les mots dans le corpus donné. Le processus d'optimisation affine les vecteurs de mots pour maximiser la vraisemblance d'observer des mots contextuels tout en minimisant la vraisemblance d'observer des exemples négatifs.

In [ ]:
#!bash get_datasets.sh ou #wget http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip

In [ ]:
import random
import numpy as np
from utils.treebank import StanfordSentiment
import matplotlib
import matplotlib.pyplot as plt
import time

import pickle
import glob
import os.path as op

In [ ]:


from utils.gradcheck import gradcheck_naive
from utils.utils import normalizeRows, softmax

In [ ]:
# Réinitialiser la graine aléatoire pour s'assurer que tout le monde obtient les mêmes résultats
dataset = StanfordSentiment()
tokens = dataset.tokens()
nWords = len(tokens)

In [ ]:
print('Longueur du vocabulaire :',nWords)

In [ ]:
dataset.sentences()[0]

In [ ]:
# Nous allons former des vecteurs à 10 dimensions pour cette mission.
dimVectors = 10

# Taille du contexte
C = 5


In [ ]:




def sigmoid(x):
    """
    Calcule la fonction sigmoïde pour l'entrée ici.
    Arguments :
    x -- Un scalaire ou un tableau numpy.
    Retourne :
    s -- sigmoïde(x)
    """
    s = 1 / (1 + np.exp(-x))
    return s


def naiveSoftmaxLossAndGradient(
    centerWordVec,
    outsideWordIdx,
    outsideVectors,
    dataset
):
    """ Fonction de perte et de gradient Softmax naïf pour les modèles word2vec

    Implémente la perte naïve softmax et les gradients entre l'intégration d'un mot central et l'intégration d'un mot extérieur.
    d'un mot central et l'intégration d'un mot extérieur. Ce sera le bloc de construction
    pour nos modèles word2vec.

    Arguments :
    centerWordVec -- numpy ndarray, intégration du mot central
                    (v_c dans le document pdf)
    outsideWordIdx -- entier, l'index du mot extérieur
                    (o de u_o dans le document pdf)
    outsideVectors -- vecteurs extérieurs (lignes de la matrice) pour tous les mots du vocabulaire
                      (U dans le document pdf)
    dataset -- nécessaire pour l'échantillonnage négatif, inutilisé ici.

    Retourne :
    loss -- perte naïve softmax
    gradCenterVec -- le gradient par rapport au vecteur central du mot
                     (dJ / dv_c dans le document pdf)
    gradOutsideVecs -- le gradient par rapport à tous les vecteurs de mots extérieurs
                    (dJ / dU)
    """


    y_hat = softmax(np.dot(centerWordVec, outsideVectors.T))
    delta = y_hat.copy()
    delta[outsideWordIdx] -= 1

    loss = -np.log(y_hat)[outsideWordIdx]
    gradCenterVec = np.dot(delta, outsideVectors)
    gradOutsideVecs = np.dot(delta[:, np.newaxis], centerWordVec[np.newaxis, :])



    return loss, gradCenterVec, gradOutsideVecs


def getNegativeSamples(outsideWordIdx, dataset, K):
    """ Échantillons K index qui ne sont pas l'outsideWordIdx """

    negSampleWordIndices = [None] * K
    for k in range(K):
        newidx = dataset.sampleTokenIdx()
        while newidx == outsideWordIdx:
            newidx = dataset.sampleTokenIdx()
        negSampleWordIndices[k] = newidx
    return negSampleWordIndices


def negSamplingLossAndGradient(
    centerWordVec,
    outsideWordIdx,
    outsideVectors,
    dataset,
    K=10
):
    """ Fonction de perte d'échantillonnage négative pour les modèles word2vec

    Implémenter la perte d'échantillonnage négative et les gradients pour un vecteur de mots centerWordVec
    et un vecteur de mots outsideWordIdx en tant que bloc de construction pour les modèles word2vec
    pour les modèles de type word2vec. K est le nombre d'échantillons négatifs à prélever.

    Remarque : le même mot peut être échantillonné négativement plusieurs fois. Par exemple
    Par exemple, si un mot extérieur est échantillonné deux fois, vous devrez
    compter deux fois le gradient par rapport à ce mot. Trois fois si
    s'il a été échantillonné trois fois, et ainsi de suite.

    Arguments/spécifications de retour : les mêmes que pour naiveSoftmaxLossAndGradient
    """


    negSampleWordIndices = getNegativeSamples(outsideWordIdx, dataset, K)
    indices = [outsideWordIdx] + negSampleWordIndices


    gradOutsideVecs = np.zeros(outsideVectors.shape)
    gradCenterVec = np.zeros(centerWordVec.shape)
    loss = 0.0
    #L = -\log(P(D = 1 | w_i, w_j)) - \sum_{k=1}^K \log(P(D = 0 | w_i, w_k))

    z = sigmoid(np.dot(outsideVectors[outsideWordIdx], centerWordVec))
    loss -= np.log(z)

    # \frac{\partial L}{\partial u(w_j)} = (P(D = 1 | w_i, w_j) - 1) \cdot v(w_i)$

    #\frac{\partial L}{\partial v(w_i)} = (P(D = 1 | w_i, w_j) - 1) \cdot u(w_j)$





    gradOutsideVecs[outsideWordIdx] += centerWordVec * (z - 1.0)
    gradCenterVec += outsideVectors[outsideWordIdx] * (z - 1.0)

    #$\frac{\partial L}{\partial u(w_k)} = -P(D = 0 | w_i, w_k) \cdot v(w_i) $

    #$ = \frac{\partial L}{\partial v(w_i)} = -P(D = 0 | w_i, w_k) \cdot u(w_k)$
    u_k = outsideVectors[negSampleWordIndices]
    z = sigmoid(-np.dot(u_k,centerWordVec))
    loss += np.sum(- np.log(z))
    gradCenterVec += np.dot((z-1),u_k)*(-1)
    gradOutsideVecs[negSampleWordIndices] += np.outer((z-1),centerWordVec)*(-1)


    return loss, gradCenterVec, gradOutsideVecs


def skipgram(currentCenterWord, windowSize, outsideWords, word2Ind,
             centerWordVectors, outsideVectors, dataset,
             word2vecLossAndGradient=naiveSoftmaxLossAndGradient):
    """ Modèle de skip-gram dans word2vec

    Implémente le modèle skip-gram dans cette fonction.

    Arguments :
    currentCenterWord -- une chaîne du mot central actuel
    windowSize -- entier, taille de la fenêtre de contexte
    outsideWords -- liste de 2*windowSize au maximum, les mots extérieurs
    word2Ind -- un dictionnaire qui associe les mots à leurs indices dans la
              dans la liste des vecteurs de mots
    centerWordVectors -- vecteurs de mots centraux (en tant que lignes) pour tous les mots du vocabulaire
                        (V dans le document pdf)
    outsideVectors -- vecteurs de mots extérieurs (sous forme de lignes) pour tous les mots du vocabulaire
                    (U dans le document pdf)
    word2vecLossAndGradient -- la fonction de perte et de gradient pour
                               un vecteur de prédiction étant donné les vecteurs de mots outsideWordIdx
                               vecteurs de mots, peut être l'une des deux fonctions de perte
                               fonctions de perte que vous avez implémentées ci-dessus.

    Retourne :
    loss -- la valeur de la fonction de perte pour le modèle skip-gram
            (J dans le document pdf)
    gradCenterVecs -- le gradient par rapport aux vecteurs de mots centraux
            (dJ / dV dans le document pdf)
    gradOutsideVectors -- le gradient par rapport aux vecteurs de mots extérieurs
                        (dJ / dU dans le document pdf)
    """

    loss = 0.0
    gradCenterVecs = np.zeros(centerWordVectors.shape)
    gradOutsideVectors = np.zeros(outsideVectors.shape)


    currentCenterWordIdx = word2Ind[currentCenterWord]
    centerWordVec = centerWordVectors[currentCenterWordIdx]

    for outsideWord in outsideWords:
        outsideWordIdx = word2Ind[outsideWord]
        (l, gradCenter, gradOutside) = word2vecLossAndGradient(
            centerWordVec, outsideWordIdx, outsideVectors, dataset)
        loss += l
        gradCenterVecs[currentCenterWordIdx] += gradCenter
        gradOutsideVectors += gradOutside

    return loss, gradCenterVecs, gradOutsideVectors



In [ ]:
windowSize=5

In [ ]:
wordVectors = np.concatenate(
    ((np.random.rand(nWords, dimVectors) - 0.5) /
       dimVectors, np.zeros((nWords, dimVectors))),
    axis=0)

In [ ]:
x0, step, iterations, postprocessing,PRINT_EVERY=wordVectors, 0.3, 40000, lambda x: x, 10

In [ ]:
ANNEAL_EVERY = 20000
start_iter=0
x = x0

In [ ]:
exploss = None
batchsize = 50
for iter in range(start_iter + 1, iterations + 1):


        loss = None

        loss = 0.0
        grad = np.zeros(wordVectors.shape)
        N = wordVectors.shape[0]
        centerWordVectors = wordVectors[:int(N/2),:]
        outsideVectors = wordVectors[int(N/2):,:]
        for i in range(batchsize):
            windowSize1 = random.randint(1, windowSize)
            centerWord, context = dataset.getRandomContext(windowSize1)

            c, gin, gout = skipgram(
                centerWord, windowSize1, context,  tokens, centerWordVectors,
                outsideVectors, dataset, negSamplingLossAndGradient
            )
            loss += c / batchsize
            grad[:int(N/2), :] += gin / batchsize
            grad[int(N/2):, :] += gout / batchsize
        x -= step * grad

        x = postprocessing(x)
        if iter % PRINT_EVERY == 0:
            if not exploss:
                exploss = loss
            else:
                exploss = .95 * exploss + .05 * loss
            print("iter %d: %f" % (iter, exploss))



In [ ]:
# concaténer les vecteurs de mots d'entrée et de sortie
wordVectors = np.concatenate(
    (wordVectors[:nWords,:], wordVectors[nWords:,:]),
    axis=0)

In [ ]:

visualizeWords = [
    "great", "cool", "brilliant", "wonderful", "well", "amazing",
    "worth", "sweet", "enjoyable", "boring", "bad", "dumb",
    "annoying", "female", "male", "queen", "king", "man", "woman", "rain", "snow",
    "hail", "coffee", "tea"]

In [ ]:

visualizeIdx = [tokens[word] for word in visualizeWords]
visualizeVecs = wordVectors[visualizeIdx, :]

In [ ]:
temp = (visualizeVecs - np.mean(visualizeVecs, axis=0))
covariance = 1.0 / len(visualizeIdx) * temp.T.dot(temp)
U,S,V = np.linalg.svd(covariance)
coord = temp.dot(U[:,0:2])

In [ ]:

for i in range(len(visualizeWords)):
    plt.text(coord[i,0], coord[i,1], visualizeWords[i],
        bbox=dict(facecolor='green', alpha=0.1))

plt.xlim((np.min(coord[:,0]), np.max(coord[:,0])))
plt.ylim((np.min(coord[:,1]), np.max(coord[:,1])))


